In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [3]:
from IPython.display import clear_output

In [4]:
!pip install shap
clear_output()

In [1]:
import sklearn

In [ ]:
import torch
import numpy as np
import pandas as pd
import pickle
import re


import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import shap

shap.initjs()

In [6]:
def replace_outliers_with_average(data):
    rows, cols = data.shape
    for col in range(cols):
        column_data = data[:, col]
        Q1 = np.quantile(column_data, 0.01)
        Q3 = np.quantile(column_data, 0.99)
        # IQR = Q3 - Q1
        lower_bound = Q1# - 1.5 * IQR
        upper_bound = Q3# + 1.5 * IQR

        # Identify outliers
        outliers_mask = (column_data < lower_bound) | (column_data > upper_bound)

        # Calculate the average of non-outliers
        non_outliers_avg = column_data[~outliers_mask].mean()
        # print(non_outliers_avg)

        # Replace outliers with the average of non-outliers
        data[outliers_mask, col] = non_outliers_avg

    return data

In [7]:
train_path = '/content/gdrive/MyDrive/speech_analysis/train_with_groups.csv'
test_path = '/content/gdrive/MyDrive/speech_analysis/test_with_groups.csv'

# Load your data
train_df = pd.read_csv(train_path, header=[0, 1]).fillna(0)
test_df = pd.read_csv(test_path, header=[0, 1]).fillna(0)

In [8]:
train_df.replace([np.inf, -np.inf], 0, inplace=True)
test_df.replace([np.inf, -np.inf], 0, inplace=True)

# Prepare the datasets
X_train = train_df.drop(["Info"], axis=1).values
y_train = train_df[("Info", 'label')].values
X_val = test_df.drop(["Info"], axis=1).values
y_val = test_df[("Info", 'label')].values

X_train = replace_outliers_with_average(X_train.copy())
X_val = replace_outliers_with_average(X_val.copy())

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

X_val = replace_outliers_with_average(X_val.copy())

<ipython-input-8-8a7b43ec8d07>:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X_train = train_df.drop(["Info"], axis=1).values
<ipython-input-8-8a7b43ec8d07>:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X_val = test_df.drop(["Info"], axis=1).values


In [9]:
mmse_train_df = pd.read_excel("/content/gdrive/MyDrive/speech_analysis/gender/train_data_2021.xlsx")
mmse_test_df = pd.read_excel("/content/gdrive/MyDrive/speech_analysis/gender/test_data_2021.xlsx")

In [10]:
train_df.columns = train_df.columns.droplevel(level = 0)
test_df.columns = test_df.columns.droplevel(level = 0)

In [11]:
train_df["filename"] = train_df["filename"].apply(lambda x: x.split(".")[0].strip())
test_df["filename"] = test_df["filename"].apply(lambda x: x.split(".")[0].strip())

In [12]:
train_mmse = pd.merge(train_df, mmse_train_df, left_on="filename", right_on="id")[["filename", "mmse"]]
test_mmse = pd.merge(test_df, mmse_test_df, left_on="filename", right_on="id")[["filename", "mmse"]]

train_mmse = train_mmse["mmse"].values
test_mmse = test_mmse["mmse"].values

all_mmse = np.concatenate((test_mmse, train_mmse))

In [13]:
feature_names = train_df.columns[:-2].tolist()
feature_names.append("MMSE")

In [14]:
feature_main = {'LOG_MEL_SPECTROGRAM': 26, 'MFCC': 15, 'LPC': 9, 'LPCC': 9, 'ENVELOPE': 16, 'PLP': 9, 'lspFreq': 8, 'msc': 13}

In [15]:
functions = ['max',
 'min',
 'span',
 'maxPos',
 'minPos',
 'amean',
 'linregc1',
 'linregc2',
 'linregerrA',
 'linregerrQ',
 'stddev',
 'skewness',
 'kurtosis',
 'quartile1',
 'quartile2',
 'quartile3',
 'iqr1_2',
 'iqr2_3',
 'iqr1_3',
 'percentile1',
 'percentile99',
 'pctlrange0_1',
 'upleveltime75',
 'upleveltime90']

In [16]:
def agg_matrices(X_train, X_val, feature_names, de=False):
    for feature in feature_main:
        for f in functions:
            if de:
                pattern = re.compile(f"^{feature}_sma_de\[\d+\]_{f}$")
            else:
                pattern = re.compile(f"^{feature}_sma\[\d+\]_{f}$")

            matching_indices = [i for i, s in enumerate(feature_names) if pattern.match(s)]
            if feature == "MFCC":
                feature_names = np.delete(feature_names , matching_indices[1:])
                X_train = np.delete(X_train, matching_indices[1:], axis=1)
                X_val = np.delete(X_val, matching_indices[1:], axis=1)
                matching_indices = matching_indices[:1]

            if (len(matching_indices) == 0):
                print(f"{feature}_sma\[\d+\]_{f}", matching_indices)
                continue

            X_train_mean_column = np.mean(X_train[:, matching_indices], axis=1)
            X_val_mean_column = np.mean(X_val[:, matching_indices], axis=1)

            insert_position = min(matching_indices)  # typically, you would want it where the first column was

            X_train = np.delete(X_train, matching_indices, axis=1)
            X_train = np.insert(X_train, insert_position, X_train_mean_column, axis=1)

            X_val = np.delete(X_val, matching_indices, axis=1)
            X_val = np.insert(X_val, insert_position, X_val_mean_column, axis=1)


            feature_names = np.delete(feature_names, matching_indices)
            name = f"{feature}_sma_de_{f}" if de else f"{feature}_sma_{f}"
            feature_names = np.insert(feature_names, insert_position, f"{feature}_sma_de_{f}")


    return X_train, X_val, feature_names

In [17]:
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")

X_train, X_val, feature_names = agg_matrices(X_train, X_val, np.array(feature_names), de=False)
X_train, X_val, feature_names = agg_matrices(X_train, X_val, np.array(feature_names), de=True)

print("After aggregating matrices:")
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")

X_train shape: (166, 6859)
X_val shape: (71, 6859)
After aggregating matrices:
X_train shape: (166, 1627)
X_val shape: (71, 1627)


In [ ]:
# models = []
# f1s = []
# for i in range(10):
#     model_1 = RandomForestClassifier(max_depth=4)
#     model_1.fit(X_train, y_train)
#     f1 = f1_score(model_1.predict(X_val), y_val)
#     print(f1)
#     models.append(model_1)
#     f1s.append(f1)
model_1 = pickle.load(open("/content/gdrive/MyDrive/speech_analysis/rf0.76pkl_mfcc", 'rb'))

In [21]:
model_1 = models[np.argmax(f1s)]

In [22]:
f1_score(model_1.predict(X_val), y_val)

0.7428571428571429

In [23]:
neww = np.concatenate((X_val, X_train), axis=0)

In [24]:
neww.shape

(237, 1627)

In [25]:
explainer = shap.Explainer(model_1)
shap_values = explainer.shap_values(neww)

In [26]:
num_samples, num_features, num_outputs = shap_values.shape
all_features = np.concatenate((neww, all_mmse.reshape(-1, 1)), axis=1)
new_feature_shap = np.zeros((237, 1, num_outputs))
shap_values_with_new_feature = np.concatenate([shap_values, new_feature_shap], axis=1)

In [27]:
shap_values = shap_values_with_new_feature[:, :, 1]

In [28]:
shap_values.shape

(237, 1628)

In [29]:
selected_features = [
    "ENVELOPE_sma_de_linregc2",
    "jitter_sma_de_iqr2_3",
    "hesitation_rate",
    "MFCC_sma_de_linregc2",
    "pause_lengths_avg",
    "num_of_sylablles",
    "pause_speech_duration_ratio",
    "F1_sma_de_quartile2",
    "mean_words_in_utterance",
    "F0_sma_de_iqr1_2",
    "PLP_sma_de_linregc1",
    "LTAS_sma_iqr1_3",
    "amp_range_sma_quartile3",
    "AMP_ENTROPY_sma_upleveltime90",
    "F2_sma_skewness",
    "MFCC_sma_de_amean",
    "LTAS_sma_span",
    "LPCC_sma_de_span",
    "NHR_sma_kurtosis",
    "jitter_sma_amean",
    "transformed_phonation_rate",
    "SHIMMER_sma_iqr1_2",
    "CPP_sma_linregc1",
    "F3_sma_linregc1",
    "LTAS_sma_stddev",
    "speech_length",
    "SPL_sma_de_kurtosis",
    "SPL_sma_de_iqr1_3",
    "relative_sentence_duration_sma_de_span",
    "regularity_8",
    "MMSE"
]


In [30]:
selected_feature_names = [
    "Spectral envelope (sma_de_linregc2)",
    "Jitter (sma_de_iqr2_3)",
    'Hesitation rate',
    'Mel-frequency cepstral coefficients (sma_de_linregc2)',
    'Average pause length',
    'Syllable count',
    'Pause length ratio',
    'F1 (sma_de_quartile2)',
    'Mean Number of words in utterances',
    'F0 (sma_de_iqr1_2)',
    'Perceptual linear predictive coefficients (sma_de_linregc1)',
    'Long term average spectrum (sma_iqr1_3)',
    'Amplitude range (sma_quartile3)',
    'Amplitude entropy (sma_upleveltime90)',
    'F2 (sma_skewness)',
    'Mel-frequency cepstral coefficients (sma_de_amean)',
    'Long term average spectrum (sma_span)',
    'Linear predictive cepstral coefficients (sma_de_span)',
    'Noise to harmonic ratio (sma_kurtosis)',
    'Jitter (sma_amean)',
    'Transformed Phonation Rate ',
    'Shimmer (sma_iqr1_2)',
    'Cepstral peak prominence (sma_linregc1)',
    'F3 (sma_linregc1)',
    'Long term average spectrum (sma_stddev)',
    'Total length of speech',
    'Sound pressure level (sma_de_kurtosis)',
    'Sound pressure level (sma_de_iqr1_3)',
    'Relative sentence duration (sma_de_span)',
    'Regularity (stddev of silence segments)',
    "MMSE"
]

In [31]:
indices = []
list_feature_names = list(feature_names)
for feature in selected_features:
    indices.append(list_feature_names.index(feature))


In [32]:
print(indices)

[817, 226, 1, 673, 5, 17, 6, 271, 16, 177, 960, 587, 392, 1602, 292, 671, 571, 764, 1207, 190, 27, 1115, 864, 335, 579, 10, 1327, 1333, 67, 37, 1627]


In [33]:
feature_names = feature_names[indices]
shap_values = shap_values[:, indices]
neww = all_features[:, indices]

In [34]:
shap.summary_plot?

In [ ]:
shap.summary_plot(shap_values, neww, feature_names=selected_feature_names, max_display=30,
                  show=False, plot_size=[16, 12])

_ = plt.yticks(fontsize=11)
_ = plt.xticks(fontsize=11)


fig, ax = plt.gcf(), plt.gca()

fig.axes[-1].set_ylabel('Feature Value', fontsize=11)
fig.axes[-1].tick_params(labelsize=8)

total_shap_values = np.abs(shap_values).sum(axis=0)

# Sort the features by total SHAP value
sorted_indices = np.argsort(total_shap_values)[::-1]
sorted_feature_names = [selected_feature_names[i] for i in sorted_indices]
sorted_total_shap_values = total_shap_values[sorted_indices]

# # Create the beeswarm plot
# # shap.summary_plot(shap_values, features, plot_type="dot", show=False)

y_tick_labels = [item.get_text() for item in ax.get_yticklabels()]
y_tick_labels.reverse()

# # Create new y-axis labels with feature names aligned to the left and total SHAP values aligned to the right
new_y_tick_labels = [
    f"{label:<0} {total:>10.2f}" for label, total in zip(sorted_feature_names, sorted_total_shap_values)
]

# # Update the y-axis labels
new_y_tick_labels.reverse()
ax.set_yticklabels(new_y_tick_labels[1:31])


_ = plt.xlabel("SHAP value (impact on model output)", fontsize=11)



axes = fig.get_axes()

# The colorbar can usually be found as the last axes object
# Adjust colorbar size and position
if len(axes) > 1:
    cbar = axes[-1]
    cbar.set_box_aspect(4)
    cbar.set_position([0.9, 0.1, 0.02, 0.8])
    cbar.tick_params(labelsize=11)


ax.tick_params(axis='y', pad=-15)
ax.grid(True, alpha=0.5)

# Show the plot with the updated y-axis labels
plt.show()

In [ ]:
fig.savefig("summary plot.svg", dpi=600)

In [ ]:
df["speech_rate_words"] = df["speech_rate_words"] * (1 / df["tlt"])
df["speech_rate_syllable"] = df["speech_rate_syllable"] * (1 / df["tlt"])

In [ ]:
columns = [
      "filename",
      "label",
      "jitter_sma_de_iqr2_3",
      "jitter_sma_amean",
      "SHIMMER_sma_iqr1_2",
      "transformed_phonation_rate",
      "AMP_ENTROPY_sma_upleveltime90",
      "LTAS_sma_iqr1_3",
      "LTAS_sma_span",
      "LTAS_sma_stddev",
      "speech_rate_words",
      "speech_rate_syllable",
      "transformed_phonation_rate",
      "pause_lengths_avg",
      "pause_speech_duration_ratio",
      "SPL_sma_de_kurtosis",
      "SPL_sma_de_iqr1_3",







]

In [ ]:
df[columns].to_csv("features.csv")